In [1]:
%matplotlib widget
from tqdm.auto import tqdm, trange
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import math
from numpy.random import default_rng, SeedSequence
import multiprocessing
import concurrent.futures
from numba import vectorize,jit,prange,objmode,njit

In [2]:
rng = default_rng()

In [3]:
error = 1e-10 #because machine error is *really* important when working with linalg

In [4]:
def normalize(v):
    l = np.linalg.norm(v)
    if l < error:
        print("Something has gone terribly wrong with "+str(v))
        return v
    return v/l

Questions for next version:
* Make precise:
    * Why probability 1? Give intuition and what measure 0 set works
    * Say what $x_n, p_n, v_n$ are, give picture
    * Conjecture 4.1: Provide evidence and convince the reader that, for irrational starting direction 
        * Get some random [convex polygons](https://cglab.ca/~sander/misc/ConvexGeneration/convex.html) and see if that works
    * Check conj 4.2 for irregular triangles
* Density of paths:
    * How to measure?
        * Maybe try making circles and finding the largest radius $\epsilon$ s.t. $B_\epsilon \cap L =\emptyset$ (for $L$ is the set of points in the lines)
    * How dense are random paths on 7-gons? Show with 1000 bounces
    * Are there protected regions in small $n$ $n$-gons? If so, how?

### Defining shapes
In general we will define a discrete shape to be an array of pairs of points which represent the lines between them.

In [5]:
def polar_poly(r,n):
    r = np.vectorize(r)
    theta = 2*np.pi/n
    angles = np.array([x*theta for x in range(n)])
    p0s = np.vstack((r(angles)*np.cos(angles),r(angles)*np.sin(angles))).T
    p1s = np.array(list(map(lambda i: p0s[(i+1)%n],range(n))))
    return np.array(list(zip(p0s,p1s)))

In [6]:
fig = plt.figure()
shape = polar_poly(lambda t: 1+np.cos(t),6)
plt.plot(*shape.T)
plt.gca().set_aspect(1)
plt.plot(0,0,'ro')
plt.axis('off')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [7]:
def ngon(n):
    return polar_poly(lambda t: 1,n)

In [8]:
fig = plt.figure()
shape = ngon(5)
plt.plot(*shape.T)
plt.gca().set_aspect(1)
plt.plot(0,0,'ro')
plt.axis('off')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
def circle_arcs(n):
    f = lambda t: np.array([np.cos(t),np.sin(t)])
    fs = []
    ns = []
    tbounds = []
    for i in range(n):
        fs.append(f)
        ns.append(f)
        tbounds.append((2*np.pi/i,2*np.pi/(i+1)))
    return (fs,ns,tbound)

In [10]:
def polar_curve(r,rprime):
    rhat = lambda t: np.array([np.cos(t),np.sin(t)])
    that = lambda t: rhat(t+np.pi/2)
    f = lambda t: r(t)*np.array([np.cos(t),np.sin(t)])
    n = lambda t: -r(t)*rhat(t)+rprime(t)*that(t)
    tbound = (0,2*np.pi)
    return ([f],[n],[tbound])

General Hit Algorithms

In [11]:
#@njit(fastmath=True,parallel=True)
def line_hit(x,v,p0,p1):
    dp = p0-p1#np.subtract(p0,p1)
    a = np.vstack((v,dp)).T
    b = p0-x#np.subtract(p0,x)
    t = np.linalg.solve(a,b)
    nx = x+t[0]*v #np.add(x,np.multiply(t[1],v))
    p = p0-t[1]*dp#np.add(p0,np.multiply(t[0],dp))
    if t[1] < 1 and t[1] > 0 and t[0] > 0 and np.linalg.norm(p-nx) < error:
        n = np.array([-dp[1],dp[0]])
        return True, (p,n,(x,v,t[0]))
    else:
        return False, ()

In [12]:
#@jit(fastmath=True,parallel=True,forceobj=True)
def curve_hit(x,v,f,n,tbound):
    g = lambda st: (x+v*st[0])-f(st[1])
    hit,_,_,i,m = fsolve[g,[1,0.5*tbound[0]+0.5*tbound[1]]]
    if i == 1:
        r = normalize(n(hit[1]))
        p = x+v*hit[0]
        return True, (p,r,(x,v,hit[1]))
    return False, ()

Filtering specific hits of piecewise curves/shapes

In [13]:
#@jit(fastmath=True,parallel=True,forceobj=True)
def general_hit(hits):
    crossings = list(map(lambda x: (x[1],x[2]), filter(lambda x: x[0], hits)))
    #crossings = hits[:][0][[1,2]]
    if crossings == []:
        raise Exception("No hits!")
    cross = sorted(crossings,key=lambda x: x[0][2][2])[0]
    p = cross[0][0]
    n = cross[0][1]
    s = cross[1]
    t = cross[0][2][2]
    return p, n, s, t

In [14]:
def polygon_hit(ps):
    def hit(x,v):
        hits = list(map(lambda p: line_hit(x,v,*p[1])+(p[0],),enumerate(ps))) #append i to the side
        return general_hit(hits)
    return hit

In [15]:
def piecewise_smooth(fs,ns,tbounds):
    def hit(x,v):
        hits = list(map(lambda ifnt: curve_hit(x,v,ifnt[1],ifnt[2],ifnt[3])+(ifnt[0],),enumerate(zip(fs,ns,tbounds))))
        return general_hit(hits)
    return hit

Special Case

In [39]:
#@njit(fastmath=True,parallel=True)
def circle_hit(x,v):
    b = np.dot(v,x)
    c = np.linalg.norm(x)**2 -1
    ts = -b+np.array([1,-1])*np.sqrt(b**2 - c)
    t = np.max(ts)#np.min(ts[np.all(np.greater(ts,[0,0]))])
    p = x+t*v
    n = -p
    return p, n, np.arctan2(x[1],x[0]), np.arctan2(x[1],x[0])

Random Convex Shapes

In [17]:
def rand_convex(n):
    if n<3:
        raise Exception("Degenerate polygons not supported! (n<3)")
    x = np.sort(rng.random(size=n)*2-1)
    y = np.sort(rng.random(size=n)*2-1)
    minx,maxx = x[0],x[-1]
    miny,maxy = y[0],y[-1]
    
    xbs = rng.choice([True,False],size=n-2)
    x0 = x[1:-1]
    x1 = x0[xbs]
    x2 = x0[np.logical_not(xbs)]
    
    ybs = rng.choice([True,False],size=n-2)
    y0 = y[1:-1]
    y1 = y0[ybs]
    y2 = y0[np.logical_not(ybs)]
    
    x1 = np.hstack((minx,x1,maxx))
    x2 = np.hstack((minx,x2,maxx))
    xs = np.zeros(n)
    for i,x in enumerate(x1[:-1]):
        xs[i] = x1[i+1]-x
    for i,x in enumerate(x2[:-1]):
        xs[i+x1.size-1] = x-x2[i+1]
        
    y1 = np.hstack((miny,y1,maxy))
    y2 = np.hstack((miny,y2,maxy))
    ys = np.zeros(n)
    for i,y in enumerate(y1[:-1]):
        ys[i] = y1[i+1]-y
    for i,y in enumerate(y2[:-1]):
        ys[i+y1.size-1] = y-y2[i+1]
        
    rng.shuffle(ys)
    
    vec = list(zip(xs,ys))
    vec.sort(key=lambda x: np.arctan2(x[1],x[0]))
    vec = np.array(vec)
    
    p = np.array([0,0])
    ps = np.zeros((n,2))
    ps[0] = p
    for i,v in enumerate(vec):
        p = p+v
        ps[i] = p
        
    del p
    off = np.array([minx,miny]) - np.min(ps,axis=0)
    p0s = np.array(list(map(lambda p: p+off,ps)))
    p1s = np.array(list(map(lambda i: p0s[(i+1)%n],range(n))))
    
    return np.array(list(zip(p0s,p1s)))

In [18]:
fig = plt.figure()
shape = rand_convex(7)
plt.plot(*shape.T)
plt.gca().set_aspect(1)
plt.plot(0,0,'ro')
plt.axis('off')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Bounce algorithm

In [19]:
#@njit(fastmath=True,parallel=True)
def bounce(v,n):
    n = normalize(n)
    return v-2*np.dot(n,v)*n#np.subtract(v,np.multiply(2*np.dot(n,v),n))

Simulation of many bounces

In [20]:
#@jit(fastmath=True,parallel=True,forceobj=True)
def sim(hitFunc,x,v,depth):
    xs = np.zeros((depth+1,2))
    vs = np.zeros((depth+1,2))
    ss = np.zeros(depth)
    ts = np.zeros(depth)
    xs[0] = x
    vs[0] = v
    ss[0] = -1
    ts[0] = -1
    for i in range(depth):
        x, n, s, t = hitFunc(x,v)
        v = bounce(v,n)
        xs[i+1] = x
        vs[i+1] = v
        ss[i] = s
        ts[i] = t
        x = x+error*v
    return xs,vs,ss,ts

In [21]:
def dynamical_sim(hitFunc,x,v,side,depth):
    xs,vs,ss,ts = sim(hitFunc,x,v,depth)
    return xs,ts[ss==side]

Random Start Info

In [22]:
def randx(rad):
    r = rng.random()*rad
    theta = 2*np.pi*rng.random()
    x = np.multiply(r,np.array([np.cos(theta),np.sin(theta)]))
    return x
def randv():
    vtheta = 2*np.pi*rng.random()
    v = np.array([np.cos(vtheta),np.sin(vtheta)])
    return v

Get data for plotting

In [23]:
def side_chance(ss,mod=0):
    if mod == 0:
        s = ss
    else:
        s = np.array(list(map(lambda s: s%mod,ss)))
    uniques = np.unique(s)
    counts = np.array(list(map(lambda u: sum(1 for side in s if side == u),uniques)))
    counts = counts/np.sum(counts)
    return uniques,counts

Dynamical Analysis

In [24]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
shape = ngon(4)#10*rand_convex(5)
shapehit = polygon_hit(shape)
ts = np.linspace(0,np.pi/2,num=1000)
for t in tqdm(ts):
    break
    _,hs = dynamical_sim(shapehit,np.array([0.25-1,0.24]),np.array([np.cos(t),np.sin(t)]),0,5000)
    xns = hs[1:]
    xn1s = hs[:-1]
    ax.plot(xns,xn1s,t,',')
    #xy = np.linspace(0,max(hs))
    #ax.plot(xy,xy,t,'r--')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Random N-Gon Sims

In [28]:
def sl(q):
    return np.linalg.norm(q[1]-q[0])
sidelen = np.vectorize(sl,signature='(2,2)->()')
#@njit(fastmath=True,parallel=True)
def lenprob(depth,sides,trials,tries=50):
    length = np.zeros((trials,sides))
    prob = np.zeros((trials,sides))
    theta = np.zeros(trials)
    
    for k in trange(trials):
        shape = 10*rand_convex(sides)
        hit = polygon_hit(shape)
        sidelengths = sidelen(shape)
        t = 0
        ls = None
        while t < tries:
            t += 1
            v = randv()
            x = randx(1)
            try:
                xs,vs,ss,_ = sim(hit,x,v,depth)
            except:
                continue
            t = tries+1
            us,cs = side_chance(ss)
            us = np.int_(us)
            length[k] = sidelengths/np.sum(sidelengths)
            prob[k][us] = cs
            theta[k] = np.arctan2(v[1],v[0])
    return length.flatten(), prob.flatten(), theta

In [29]:
plt.figure()
#l,p,_ = lenprob(1000,100,2000,tries=1)
fit = np.polyfit(l,p,1)
poly = np.poly1d(fit)
lenth = np.linspace(0,max(l))
print("{}x+{}".format(fit[0],fit[1]))
plt.plot(l,p,'k,',label="100-gons")
plt.plot(lenth,poly(lenth),'r--',label="Best Fit")
plt.legend()
plt.savefig('lenprob100.png')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

NameError: name 'l' is not defined

In [30]:
fig = plt.figure()
nums = [3,4,5,6,7,8,9]
colors = "krbgcmy"
for n,c in zip(nums,colors):
    l,p,_ = lenprob(2000,n,3000)
    plt.plot(l,p,c+',',label="{}-gons".format(n))
    fit = np.polyfit(l,p,1)
    print("{}x+{}".format(fit[0],fit[1]))
plt.legend()
plt.gca().set_aspect(1)
plt.savefig('lenprob.png')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …


0.9999614552622299x+5.002250412879494e-06



1.000148498492592x+-2.3908257307224805e-05



1.0008269600365x+-0.00013380213390566358



0.9995029396402034x+7.475235522038706e-05



0.9988929290384156x+0.0001504034977810594



0.9993006340454906x+8.538092694636369e-05



1.0000043508242262x+-4.798797978326074e-07


In [ ]:
#from mpl_toolkits.mplot3d import Axes3D
#fig = plt.figure()
#ax = fig.add_subplot(111, projection='3d')
#ax.scatter(l,p,a)
#plt.savefig('lenprobangle.png')
#plt.show()

Specific Square Sims

In [43]:
sq = ngon(4)
sqhit = polygon_hit(sq)
x0 = np.array([0.25-1,0.24])
slopes = [np.sqrt(2)/2,np.sqrt(3),np.sqrt(5)]
n = 100
for m in slopes:
    fig = plt.figure()
    ps,vs,ss,_ = sim(sqhit,x0,np.array([m,1]),n)
    plt.plot(*sq.T)
    xs = ps[:,0]
    ys = ps[:,1]
    plt.plot(xs,ys)
    x = ps[-1]
    v = vs[-1]
    plt.quiver(*x,*v)
    plt.gca().set_aspect(1)
    plt.axis('off')
    plt.savefig('{}for{}.png'.format(np.round(m,3),n))
    plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<ipython-input-43-e195e7835e77>:7: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig = plt.figure()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [44]:
fig = plt.figure()
#print(ss)
ax = plt.gca()
ax.hist(ss,bins=2*4)
us,cs = side_chance(ss,mod=2)
plt.bar(us,cs)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<BarContainer object of 2 artists>

Specifically Circular Sims

In [45]:
fig = plt.figure()
deg = 30
theta = np.pi/2-(np.pi/180)*deg #<- Edit this!
ps,vs,ss,ts = sim(circle_hit,np.array([0,1]),np.array([np.cos(theta),np.sin(theta)]),1000)
xs = ps[:,0]
ys = ps[:,1]
plt.plot(xs,ys)
x,v, = ps[-1],vs[-1]
plt.quiver(*x,*v)
draw_circle = plt.Circle((0, 0), 1,fill=False)
plt.xlim(-1,1)
plt.ylim(-1,1)
plt.gcf().gca().add_artist(draw_circle)
plt.gca().set_aspect(1)
plt.axis('off')
plt.savefig("{}deg.png".format(deg))
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [46]:
fig = plt.figure()
#print(ss) #<- Side sequence
#ax = plt.gca()
#ax.hist(ss,bins=bounces)
us,cs = side_chance(ss)
plt.bar(us,cs,width=0.05)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<BarContainer object of 998 artists>

Discrete Cardiod Sims

In [56]:
fig = plt.figure()
shape = polar_poly(lambda t: 1+np.cos(t),100)
shapehit = polygon_hit(shape)
ps,vs,ss,ts = sim(shapehit,randx(0.5)+np.array([0.5,0]),randv(),10000)
plt.plot(*shape.T)
xs = ps[:,0]
ys = ps[:,1]
plt.plot(xs,ys)
x,v = ps[-1],vs[-1]
plt.quiver(*x,*v)
ax = plt.gca()
ax.set_aspect(1)
plt.axis('off')
plt.savefig('100cardiodrand.png')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [57]:
fig = plt.figure()
#print(ss)
#ax = plt.gca()
#ax.hist(ss,bins=bounces)
us,cs = side_chance(ss)
plt.bar(us,cs,width=0.75)
plt.savefig("6normcardiodsidesplot.png")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [67]:
fig = plt.figure()
#(fs,ns,ts) = polar_curve(lambda t: 1+np.cos(t),lambda t: -np.sin(t))
#hit = piecewise_smooth(fs,ns,ts)
#ls = sim(hit,randx(0.5)+np.array([0.75,0]),randv(),1000)
ts = np.linspace(0,2*np.pi)
xs = (1+np.cos(ts))*np.cos(ts)
ys = (1+np.cos(ts))*np.sin(ts)
plt.plot(xs,ys)
#xs = list(map(lambda x: x[0][0], ls))
#ys = list(map(lambda x: x[0][1], ls))
#plt.plot(xs,ys)
#x,v,_ = ls[-1]
#plt.quiver(*x,*v)
ax = plt.gca()
ax.set_aspect(1)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Setting up shapes etc.

In [59]:
sides = 110
bounces = 1000
ng = ngon(sides)
nghit = polygon_hit(ng)

Random Circle & N-Gon

In [61]:
fig = plt.figure()
ps,vs,ss,ts = sim(nghit,randx(np.cos(np.pi/sides)),randv(),bounces)
plt.plot(*ng.T)
xs = ps[:,0]
ys = ps[:,1]
plt.plot(xs,ys)
x,v, = ps[-1],vs[-1]
plt.quiver(*x,*v)
plt.gca().set_aspect(1)
plt.axis('off')
plt.savefig('ngonirrational2.png')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [64]:
fig = plt.figure()
#print(ss)
#ax = plt.gca()
#ax.hist(ss,bins=2*sides)
us,cs = side_chance(ss)
plt.bar(us,cs)
plt.savefig('100.png')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [65]:
fig = plt.figure()
xs,vs,ss,ts = sim(circle_hit,randx(0.9),randv(),bounces)
xs = ps[:,0]
ys = ps[:,1]
plt.plot(xs,ys)
x,v, = ps[-1],vs[-1]
plt.quiver(*x,*v)
draw_circle = plt.Circle((0, 0), 1,fill=False)
plt.gcf().gca().add_artist(draw_circle)
plt.gca().set_aspect(1)
plt.axis('off')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [66]:
fig = plt.figure()
#print(ss)
#ax = plt.gca()
#ax.hist(ss,bins=bounces)
us,cs = side_chance(ss)
plt.bar(us,cs,width=0.05)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<BarContainer object of 1000 artists>